



# Demodulation with mean

Note: https://stackoverflow.com/questions/26171230/matplotlib-seaborn-barplot-strings-in-x-axis

Load libraries

In [ ]:
from common_imports import *

### Attacking blocking cards inside a wallet

The attack consists in iterating throught the reader messages,checking if the tag has replied and taking the mean of multiple equal samples (e.g. 5, 10, 15, 20)

All possible positions in wallet <card,blck_card>:
- 2,1
- 3,1
- 4,1


- 1,2
- 3,2
- 4,2


- 1,3
- 2,3
- 4,3


- 1,4
- 2,4
- 3,4

Set the blocking cards list and folder name

In [ ]:
from data_const import *

Iterate and make demodulation stats based on passed mean [0, 4, 9, 14] and store a csv

In [ ]:
for blck_name in blck_list:
    directory = folder_name+blck_name
    directory_results = results_folder_name+blck_name
    print(os.listdir(directory))
    for filename in os.listdir(directory): 
        file_name = os.path.join(directory, filename)
        if filename.endswith(".raw"):
            print(file_name)
            for mean_value in [0, 1, 3, 4, 7, 9, 14, 15, 31]:
            #for mean_value in [0
                stats_file = os.path.join(directory_results, filename.replace(".raw","")+"_rtl_sdr_%i.csv" % mean_value)
                print(stats_file)
                if not Path(stats_file).is_file():
                    get_demodulation_stats(
                        gnuradio_file=file_name, 
                        libnfc_file=file_name.replace(".raw",".txt"), 
                        expected_file="../data/ev1_clean/clean_4_na.txt",
                        output_file=None,
                        stats_file = stats_file,
                        mean_samples = mean_value,
                    )
                else:
                    print("already ran for", file_name)
    print("\n")

Create the dataframe to collect all stats

In [ ]:
huge_df = pd.DataFrame()
for blck_name in blck_list:
    directory_results = results_folder_name+blck_name
    for filename in os.listdir(directory_results):
        if filename.endswith(".csv"):
            df = pd.read_csv(directory_results+"/"+filename)
            # add filename, blocking card type, positions
            position = filename[:3]
            uid = blck_name + "_" + position + '_%i' % df['means'][0]
            df.insert(0,"id",uid,True)
            df.insert(0,"position",position,True)
            df.insert(0,"blck_name",blck_name,True)
            huge_df = huge_df.append(df)
##tag
# fix detected
huge_df['*_both_tag_detected'] = huge_df.apply(lambda row: row['correct_libnfc_rtlsdr'] if row['rtl_tag_detected'] < row['correct_libnfc_rtlsdr'] else row['rtl_tag_detected'], axis=1)

#1. rtl is correct, libnfc is wrong
#correct_rtlsdr_wrong_libnfc
#2. rtl is correct, libnfc is correct
# rtl_tag_correct - correct_rtlsdr_wrong_libnfc
huge_df['4_both_correct'] = huge_df['correct_libnfc_rtlsdr'] - huge_df['correct_rtlsdr_wrong_libnfc'] - huge_df['correct_libnfc_wrong_rtlsdr']
#3. rtl is wrong, libnfc is correct
# correct_libnfc_wrong_rtlsdr
# 4. detected - (somma)
huge_df['2_both_wrong'] = huge_df['*_both_tag_detected'] - huge_df['correct_libnfc_rtlsdr']
# 5. no reply = tag_expected_tot - (4)
huge_df['1_no_reply'] = huge_df['tag_expected_tot'] - huge_df['*_both_tag_detected']

huge_df['5_correct_rtlsdr_wrong_libnfc'] = huge_df['correct_rtlsdr_wrong_libnfc']
huge_df['3_correct_libnfc_wrong_rtlsdr'] = huge_df['correct_libnfc_wrong_rtlsdr']
huge_df['tot_check'] = huge_df['5_correct_rtlsdr_wrong_libnfc'] + huge_df['4_both_correct'] + huge_df['3_correct_libnfc_wrong_rtlsdr'] + huge_df['2_both_wrong'] + huge_df['1_no_reply']

huge_df['success'] = huge_df['5_correct_rtlsdr_wrong_libnfc'] + huge_df['4_both_correct'] + huge_df['3_correct_libnfc_wrong_rtlsdr']
huge_df['failed'] = huge_df['1_no_reply'] + huge_df['2_both_wrong']
huge_df['success_rate'] = huge_df['success'] / (huge_df['failed'] + huge_df['success'] )

# huge_df['correct_both'] = huge_df['rtl_tag_correct'] - (huge_df['libnfc_tag_correct'] - huge_df['correct_libnfc_wrong_rtlsdr'])
# huge_df['rtl_libnfc_tag_wrong'] = huge_df['rtl_tag_detected_both'] - huge_df['rtl_tag_correct'] - huge_df['correct_libnfc_wrong_rtlsdr']
# huge_df['rtl_tag_noreply'] = huge_df['tag_expected_tot'] - huge_df['rtl_tag_detected_both']
##reader
# huge_df['rtl_reader_wrong'] = huge_df['rtl_reader_tot'] - huge_df['rtl_reader_correct']
huge_df = huge_df.set_index('id')
huge_df = huge_df.sort_values(by=['position', 'means'])
display(huge_df)
print("saving df to %s" % (results_folder_name + "huge_df.csv"))
huge_df.to_csv(results_folder_name + "huge_df.csv")